# 03 · Exploratory Data Analysis

This notebook performs comprehensive EDA on the DNA sequence data using DuckDB for efficient queries.

## Analysis Sections
1. Basic statistics and data quality
2. Barcode sequence patterns
3. UUID distribution analysis
4. Data completeness assessment


In [ ]:
# Setup and connect to database
import duckdb
import pandas as pd
import os
from pathlib import Path

root = Path("/home/mch/dna")
db_path = root / "artifacts" / "dna.duckdb"

# Connect to database
con = duckdb.connect(str(db_path))
print(f"Connected to: {db_path}")

# Verify tables/views
tables = con.sql("SHOW TABLES").df()
print(f"\nAvailable views: {list(tables['name'])}")

## 1. Basic Statistics

In [ ]:
# Overall statistics
print("=== OVERALL STATISTICS ===")
con.sql("SELECT * FROM barcode_stats").show()

In [ ]:
# Sample of data
print("=== SAMPLE DATA (10 rows) ===")
con.sql("""
    SELECT id, barcode, LENGTH(barcode) as barcode_len
    FROM clusters
    WHERE barcode IS NOT NULL AND barcode != ''
    LIMIT 10
""").show()

## 2. Barcode Analysis

In [ ]:
# Barcode length distribution
print("=== BARCODE LENGTH DISTRIBUTION ===")
con.sql("""
    SELECT barcode_length, count, 
           ROUND(100.0 * count / SUM(count) OVER (), 2) as percentage
    FROM barcode_lengths
    LIMIT 15
""").show()

In [ ]:
# Most common barcodes
print("=== TOP 10 MOST FREQUENT BARCODES ===")
con.sql("""
    SELECT barcode, COUNT(*) as frequency,
           ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM clusters WHERE barcode IS NOT NULL), 4) as percentage
    FROM clusters
    WHERE barcode IS NOT NULL AND barcode != ''
    GROUP BY barcode
    ORDER BY frequency DESC
    LIMIT 10
""").show()

In [ ]:
# Nucleotide composition
print("=== NUCLEOTIDE COMPOSITION IN BARCODES ===")
con.sql("""
    WITH nucleotide_counts AS (
        SELECT 
            SUM(LENGTH(barcode) - LENGTH(REPLACE(barcode, 'A', ''))) as A_count,
            SUM(LENGTH(barcode) - LENGTH(REPLACE(barcode, 'C', ''))) as C_count,
            SUM(LENGTH(barcode) - LENGTH(REPLACE(barcode, 'G', ''))) as G_count,
            SUM(LENGTH(barcode) - LENGTH(REPLACE(barcode, 'T', ''))) as T_count,
            SUM(LENGTH(barcode)) as total_bases
        FROM clusters
        WHERE barcode IS NOT NULL AND barcode != ''
    )
    SELECT 
        A_count, 
        ROUND(100.0 * A_count / total_bases, 2) as A_pct,
        C_count,
        ROUND(100.0 * C_count / total_bases, 2) as C_pct,
        G_count,
        ROUND(100.0 * G_count / total_bases, 2) as G_pct,
        T_count,
        ROUND(100.0 * T_count / total_bases, 2) as T_pct,
        total_bases
    FROM nucleotide_counts
""").show()

## 3. UUID Analysis

In [ ]:
# Check UUID format compliance
print("=== UUID FORMAT ANALYSIS ===")
con.sql("""
    SELECT 
        COUNT(*) as total_ids,
        COUNT(CASE WHEN id ~ '^[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}$' 
                   THEN 1 END) as valid_uuid_format,
        COUNT(CASE WHEN id NOT SIMILAR TO '[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}' 
                   THEN 1 END) as invalid_uuid_format
    FROM clusters
""").show()

In [ ]:
# Check for duplicate IDs
print("=== DUPLICATE ID CHECK ===")
con.sql("""
    WITH id_counts AS (
        SELECT id, COUNT(*) as count
        FROM clusters
        GROUP BY id
        HAVING COUNT(*) > 1
    )
    SELECT 
        COUNT(*) as duplicate_ids,
        SUM(count) as total_duplicate_rows
    FROM id_counts
""").show()

## 4. Data Quality Summary

In [ ]:
# Comprehensive data quality report
print("=== DATA QUALITY REPORT ===\n")

# Get stats
stats = con.sql("SELECT * FROM barcode_stats").df().iloc[0]

print(f"Total Records: {stats['total_rows']:,}")
print(f"Unique Barcodes: {stats['unique_barcodes']:,}")
print(f"Records with Barcode: {stats['valid_barcodes']:,} ({100*stats['valid_barcodes']/stats['total_rows']:.1f}%)")
print(f"Records without Barcode: {stats['null_barcodes']:,} ({100*stats['null_barcodes']/stats['total_rows']:.1f}%)")

# Barcode uniqueness
if stats['valid_barcodes'] > 0:
    duplication_rate = 1 - (stats['unique_barcodes'] / stats['valid_barcodes'])
    print(f"\nBarcode Duplication Rate: {100*duplication_rate:.2f}%")
    print(f"Average occurrences per barcode: {stats['valid_barcodes']/stats['unique_barcodes']:.2f}")

In [ ]:
# Export summary statistics to CSV (optional)
# summary_df = con.sql("SELECT * FROM barcode_stats").df()
# summary_df.to_csv(root / "artifacts" / "data_summary.csv", index=False)
# print("Summary exported to artifacts/data_summary.csv")

In [ ]:
# Close database connection
con.close()
print("\nAnalysis complete!")